# 🚀 Entraînement SAC sur Kaggle

Ce notebook permet d'entraîner les agents SAC pour le trading EUR/USD.

## Configuration

**Avant de commencer :**

1. ✅ Ajoutez votre dataset `trading-data` (contenant processed_data.h5)
2. ✅ Ajoutez votre dataset `trading-sac-backend` (contenant les fichiers backend)
3. ✅ Activez le GPU (Settings → Accelerator → GPU T4 x2)
4. ✅ Uploadez le fichier `train_sac_kaggle.py` dans ce notebook

## 1️⃣ Installation des Dépendances

In [ ]:
# Installer les bibliothèques nécessaires (si pas déjà installées)
!pip install -q gymnasium h5py

## 2️⃣ Vérification de l'Environnement

In [ ]:
import os
import sys
import torch

# Vérifier GPU
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Vérifier la structure des fichiers
print("\n📁 Structure des fichiers:")
print("\nInput data:")
!ls -lh /kaggle/input/

print("\nWorking directory:")
!ls -lh /kaggle/working/

## 3️⃣ Configuration des Chemins

In [ ]:
# ⚠️ IMPORTANT: Ajustez ces chemins selon vos datasets Kaggle

# Chemin vers le fichier h5 (vérifiez le nom exact de votre dataset)
H5_PATH = "/kaggle/input/trading-data/processed_data.h5"

# Chemin vers les fichiers backend (vérifiez le nom exact de votre dataset)
BACKEND_PATH = "/kaggle/input/trading-sac-backend/backend"

# Vérifier que les fichiers existent
print(f"✅ H5 file exists: {os.path.exists(H5_PATH)}")
print(f"✅ Backend exists: {os.path.exists(BACKEND_PATH)}")

if not os.path.exists(H5_PATH):
    print("\n⚠️ ERREUR: Fichier h5 introuvable!")
    print("Vérifiez le chemin exact avec:")
    !ls -R /kaggle/input/

if not os.path.exists(BACKEND_PATH):
    print("\n⚠️ ERREUR: Dossier backend introuvable!")
    print("Vérifiez le chemin exact avec:")
    !ls -R /kaggle/input/

## 4️⃣ Copie des Fichiers Backend

In [ ]:
# Copier les fichiers backend dans le working directory
!cp -r {BACKEND_PATH} /kaggle/working/backend

# Vérifier
print("✅ Fichiers backend copiés:")
!ls -lh /kaggle/working/backend/

# Ajouter au path Python
sys.path.insert(0, '/kaggle/working')
print("\n✅ Path configuré")

## 5️⃣ Test d'Import

In [ ]:
# Tester les imports
try:
    from backend.data_pipeline import DataPipeline
    from backend.feature_engineering import FeaturePipeline
    from backend.trading_env import TradingEnvironment, TradingEnvConfig
    from backend.sac_agent import SACAgent, SACConfig
    print("✅ Tous les imports fonctionnent!")
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("\nVérifiez que tous les fichiers backend sont présents.")

## 6️⃣ Configuration de l'Entraînement

In [ ]:
# ⚙️ PARAMÈTRES D'ENTRAÎNEMENT - Ajustez selon vos besoins

CONFIG = {
    'h5_path': H5_PATH,
    'output_dir': '/kaggle/working',
    'num_episodes': 100,           # 100 pour test, 500+ pour production
    'eval_frequency': 10,          # Évaluer tous les 10 épisodes
    'checkpoint_frequency': 5,     # Sauvegarder tous les 5 épisodes
    'agent_id': 1,                 # 1, 2, ou 3
    'device': 'auto'               # 'auto', 'cuda', ou 'cpu'
}

print("⚙️ Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

## 7️⃣ Lancement de l'Entraînement

### Méthode 1: Via KaggleTrainer (Recommandé)

In [ ]:
from train_sac_kaggle import KaggleTrainer

# Créer le trainer
trainer = KaggleTrainer(**CONFIG)

# Lancer l'entraînement
print("🚀 Démarrage de l'entraînement...\n")
agent, stats = trainer.run_training()

print("\n✅ Entraînement terminé!")

### Méthode 2: Via Script Shell (Alternative)

In [ ]:
# Alternative: lancer via ligne de commande
!python train_sac_kaggle.py \
    --h5-path {CONFIG['h5_path']} \
    --output-dir {CONFIG['output_dir']} \
    --num-episodes {CONFIG['num_episodes']} \
    --eval-frequency {CONFIG['eval_frequency']} \
    --checkpoint-frequency {CONFIG['checkpoint_frequency']} \
    --agent-id {CONFIG['agent_id']} \
    --device {CONFIG['device']}

## 8️⃣ Visualisation des Résultats

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

# Charger les statistiques
with open('/kaggle/working/logs/training_stats.json', 'r') as f:
    stats = json.load(f)

# Créer les graphiques
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('🚀 SAC Training Results', fontsize=16, fontweight='bold')

# 1. Episode Rewards
axes[0, 0].plot(stats['episode_rewards'], alpha=0.6, label='Raw')
if len(stats['episode_rewards']) > 10:
    window = min(10, len(stats['episode_rewards']) // 5)
    smoothed = np.convolve(stats['episode_rewards'], np.ones(window)/window, mode='valid')
    axes[0, 0].plot(range(window-1, len(stats['episode_rewards'])), smoothed, 
                   linewidth=2, label=f'MA-{window}', color='red')
axes[0, 0].set_title('Episode Rewards')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Reward')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Episode Lengths
axes[0, 1].plot(stats['episode_lengths'], alpha=0.8, color='green')
axes[0, 1].set_title('Episode Lengths')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Steps')
axes[0, 1].grid(True, alpha=0.3)

# 3. Eval Rewards
if stats['eval_rewards']:
    eval_episodes = [i * CONFIG['eval_frequency'] for i in range(len(stats['eval_rewards']))]
    axes[0, 2].plot(eval_episodes, stats['eval_rewards'], 'o-', linewidth=2, 
                   markersize=8, color='purple')
    axes[0, 2].set_title('Validation Rewards')
    axes[0, 2].set_xlabel('Episode')
    axes[0, 2].set_ylabel('Eval Reward')
    axes[0, 2].grid(True, alpha=0.3)
else:
    axes[0, 2].text(0.5, 0.5, 'No eval data yet', 
                   ha='center', va='center', transform=axes[0, 2].transAxes)
    axes[0, 2].set_title('Validation Rewards')

# 4. Actor Loss
axes[1, 0].plot(stats['actor_losses'], alpha=0.6, color='blue')
axes[1, 0].set_title('Actor Loss')
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].grid(True, alpha=0.3)

# 5. Critic Loss
axes[1, 1].plot(stats['critic_losses'], alpha=0.6, color='orange')
axes[1, 1].set_title('Critic Loss')
axes[1, 1].set_xlabel('Episode')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].grid(True, alpha=0.3)

# 6. Alpha (Entropy Coefficient)
axes[1, 2].plot(stats['alpha_values'], alpha=0.8, color='red')
axes[1, 2].set_title('Alpha (Entropy Coefficient)')
axes[1, 2].set_xlabel('Episode')
axes[1, 2].set_ylabel('Alpha')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Graphiques sauvegardés: /kaggle/working/training_results.png")

## 9️⃣ Statistiques Finales

In [ ]:
# Calculer les statistiques finales
print("📊 STATISTIQUES FINALES")
print("="*50)

print(f"\n🎯 Récompenses:")
print(f"   Moyenne totale: {np.mean(stats['episode_rewards']):.2f}")
print(f"   Écart-type: {np.std(stats['episode_rewards']):.2f}")
print(f"   Min: {np.min(stats['episode_rewards']):.2f}")
print(f"   Max: {np.max(stats['episode_rewards']):.2f}")

if len(stats['episode_rewards']) >= 10:
    print(f"\n   Moyenne (10 derniers): {np.mean(stats['episode_rewards'][-10:]):.2f}")
    print(f"   Moyenne (10 premiers): {np.mean(stats['episode_rewards'][:10]):.2f}")
    improvement = np.mean(stats['episode_rewards'][-10:]) - np.mean(stats['episode_rewards'][:10])
    print(f"   Amélioration: {improvement:.2f} ({improvement/abs(np.mean(stats['episode_rewards'][:10]))*100:.1f}%)")

if stats['eval_rewards']:
    print(f"\n🔍 Validation:")
    print(f"   Meilleure récompense: {np.max(stats['eval_rewards']):.2f}")
    print(f"   Moyenne validation: {np.mean(stats['eval_rewards']):.2f}")

print(f"\n🧮 Pertes:")
print(f"   Actor loss (final): {stats['actor_losses'][-1]:.4f}")
print(f"   Critic loss (final): {stats['critic_losses'][-1]:.4f}")
print(f"   Alpha (final): {stats['alpha_values'][-1]:.4f}")

print(f"\n📁 Longueur épisodes:")
print(f"   Moyenne: {np.mean(stats['episode_lengths']):.0f} steps")
print(f"   Min: {np.min(stats['episode_lengths'])} steps")
print(f"   Max: {np.max(stats['episode_lengths'])} steps")

print("\n" + "="*50)

## 🔟 Liste des Fichiers Générés

In [ ]:
print("📦 Fichiers disponibles pour téléchargement:\n")

print("\n⭐ MODÈLES PRINCIPAUX:")
!ls -lh /kaggle/working/agent_*.pt 2>/dev/null || echo "   (Aucun modèle trouvé)"

print("\n💾 CHECKPOINTS:")
!ls -lh /kaggle/working/checkpoints/*.pt 2>/dev/null || echo "   (Aucun checkpoint trouvé)"

print("\n📊 LOGS ET MÉTRIQUES:")
!ls -lh /kaggle/working/logs/ 2>/dev/null || echo "   (Aucun log trouvé)"

print("\n📈 GRAPHIQUES:")
!ls -lh /kaggle/working/*.png 2>/dev/null || echo "   (Aucun graphique trouvé)"

print("\n💡 Tip: Cliquez sur 'Save Version' en haut à droite pour télécharger tous les fichiers!")

## 📦 Compression des Fichiers

In [ ]:
# Compresser tous les fichiers importants
!cd /kaggle/working && zip -r sac_trained_models.zip \
    agent_*.pt \
    checkpoints/ \
    logs/ \
    *.png \
    2>/dev/null

print("✅ Fichiers compressés!")
!ls -lh /kaggle/working/sac_trained_models.zip

## ✅ Prochaines Étapes

1. **Télécharger les modèles:**
   - Cliquez sur "Save Version" → "Save & Run All"
   - Allez dans "Output" (panneau de droite)
   - Téléchargez `agent_X_best.pt` et `sac_trained_models.zip`

2. **Sur votre machine locale:**
   ```python
   from backend.sac_agent import SACAgent, SACConfig
   
   config = SACConfig(state_dim=30, action_dim=1)
   agent = SACAgent(config=config, agent_id=1)
   agent.load("agent_1_best.pt")
   ```

3. **Tester le modèle:**
   - Utilisez les données de test
   - Évaluez les performances
   - Comparez avec les benchmarks

4. **Production:**
   - Intégrer dans votre système de trading
   - Monitoring en temps réel
   - Re-entraîner périodiquement

---

**🎉 Félicitations ! Votre agent SAC est maintenant entraîné !**